In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import warnings
from scipy.stats import ttest_ind
warnings.filterwarnings("ignore")
import os
from statsmodels.stats.proportion import proportions_ztest
import numpy as np
import scipy.stats as stats
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

# Read file

In [2]:
df_raw = pd.read_excel('UBER.xlsx', sheet_name ='Switchbacks' )

In [3]:
df_raw.head()

,city_id,period_start,wait_time,treat,commute,trips_pool,trips_express_pool,rider_cancellations,total_driver_payout,total_matches,total_double_matches
0,Boston,2018-02-19 07:00:00,2 mins,False,True,1415,3245,256,34458.411634,3372,1476
1,Boston,2018-02-19 09:40:00,5 mins,True,False,1461,2363,203,29764.349821,2288,1275
2,Boston,2018-02-19 12:20:00,2 mins,False,False,1362,2184,118,27437.367363,2283,962
3,Boston,2018-02-19 15:00:00,5 mins,True,True,1984,3584,355,44995.452993,4035,2021
4,Boston,2018-02-19 17:40:00,2 mins,False,False,1371,2580,181,27583.955295,2200,979


In [4]:
df_raw['total_ride'] = df_raw['trips_express_pool'] + df_raw['trips_pool']
df_raw['cost_per_ride'] = df_raw['total_driver_payout']/df_raw['total_ride']
df_raw['rider_cancellations_rate'] = df_raw['rider_cancellations']/df_raw['total_ride']

**Question 1 (2 Points)**.	Write a function to test the significance of the difference between the average cost per ride for a 2-minute versus a 5-minute wait time. Write hypotheses and draw conclusions based on results at 95% confidence interval.

In [5]:
from scipy.stats import ttest_ind

In [6]:
df_raw.groupby('wait_time')['total_driver_payout'].agg(['mean', 'std', 'count']).reset_index()

,wait_time,mean,std,count
0,2 mins,29291.238922,5675.727426,63
1,5 mins,27183.202390,5043.154791,63


In [7]:
group_2_min = df_raw[df_raw['wait_time'] == '2 mins']['cost_per_ride']
group_5_min = df_raw[df_raw['wait_time'] == '5 mins']['cost_per_ride']

In [8]:
## 2-sided
def test_avg_cost_difference(group_2_min, group_5_min):
    
    t_stat, p_value = ttest_ind(group_2_min, group_5_min, equal_var=False, alternative='two-sided') 

    print(f"T-Statistic: {t_stat:.4f}, P-Value: {p_value:.4f}")

    if p_value < 0.05: 
        conclusion = "Reject the null hypothesis"
    else:
        conclusion = "Fail to reject the null hypothesis."
    
    return t_stat, p_value, conclusion    

In [9]:
## 2-sided
test_avg_cost_difference(group_2_min, group_5_min)

T-Statistic: 3.4621, P-Value: 0.0007


(3.4621410571047204, 0.0007361372933601304, 'Reject the null hypothesis')

### 2-sided test hypothesis:

- State your hypothesis (H0 & H1)

Null Hypothesis (H0): There is no difference between the average cost per ride for a 2-minute versus a 5-minute wait time

Alternative Hypothesis (H1): There is a significant difference between the average cost per ride for a 2-minute versus a 5-minute wait time

- Set the level of significant (α)

assume the level of significant is 5% (at 95% confidence level)

- Drive test value by using proper statistical test

t-test

- Draw conclusion

As we have a P-Value = 0.0007, we reject the null hypothesis, indicating that the average cost per ride is significantly different between 2-minute wait compared to 5-minute wait 

-----------------------------------------------------.

In [10]:
## one-sided
def test_avg_cost_difference_1_side(group_2_min, group_5_min):
    
    t_stat, p_value = ttest_ind(group_2_min, group_5_min, equal_var=False, alternative='greater') 

    print(f"T-Statistic: {t_stat:.4f}, P-Value: {p_value:.4f}")

    if p_value < 0.05: 
        conclusion = "Reject the null hypothesis"
    else:
        conclusion = "Fail to reject the null hypothesis."
    
    return t_stat, p_value, conclusion    

In [11]:
## 1-sided
test_avg_cost_difference_1_side(group_2_min, group_5_min)

T-Statistic: 3.4621, P-Value: 0.0004


(3.4621410571047204, 0.0003680686466800652, 'Reject the null hypothesis')

### 1-sided test hypothesis:

- State your hypothesis (H0 & H1)

Null Hypothesis (H0): The average cost per ride for a 2-minute wait time is lower or equal to the one for a 5-minute wait time

Alternative Hypothesis (H1): The average cost per ride is higher for 2-minute wait compared to 5-minute wait times

- Set the level of significant (α)

assume the level of significant is 5% (at 95% confidence level)

- Drive test value by using proper statistical test

t-test

- Draw conclusion

As we have a P-Value = 0.0004, we Reject the null hypothesis, indicating that the average cost per ride is significantly higher for 2-minute wait times compared to 5-minute wait times

**Question 2(2 Points)**. Use the same function to test the significance of the difference between the average cost per ride for a 2-minute versus a 5-minute wait time during commute and non-commute hours. Write hypotheses and draw conclusions based on results at 95% confidence interval.

- Commute Hours: Compare costs for 2-min vs. 5-min wait during commute hours

- Non-Commute Hours: Compare costs for 2-min vs. 5-min wait during non-commute hours

### commute

In [12]:
group_2_min = df_raw[df_raw['wait_time'] == '2 mins']
group_5_min = df_raw[df_raw['wait_time'] == '5 mins']
group_2_min_commute = group_2_min[group_2_min['commute'] == True]
group_2_min_non_commute = group_2_min[group_2_min['commute'] == False]
group_5_min_commute = group_5_min[group_5_min['commute'] == True]
group_5_min_non_commute = group_5_min[group_5_min['commute'] == False]

In [13]:
group_2_min.groupby('commute')['cost_per_ride'].agg(['mean', 'std', 'count']).reset_index()

,commute,mean,std,count
0,False,7.280393,0.595932,53
1,True,7.812577,0.429094,10


In [14]:
group_5_min.groupby('commute')['cost_per_ride'].agg(['mean', 'std', 'count']).reset_index()

,commute,mean,std,count
0,False,6.877672,0.575014,53
1,True,7.569325,0.546833,10


In [15]:
commute_2 = group_2_min_commute['cost_per_ride']
commute_5 = group_5_min_commute['cost_per_ride']

In [16]:
def test_avg_cost_difference(commute_2, commute_5):
    
    t_stat, p_value = ttest_ind(commute_2, commute_5, equal_var=False, alternative='two-sided') 

    print(f"T-Statistic: {t_stat:.4f}, P-Value: {p_value:.4f}")

    if p_value < 0.05: 
        conclusion = "Reject the null hypothesis"
    else:
        conclusion = "Fail to reject the null hypothesis."
    
    return t_stat, p_value, conclusion    

In [17]:
test_avg_cost_difference(commute_2, commute_5)

T-Statistic: 1.1067, P-Value: 0.2838


(1.106665299623298, 0.28382083200834396, 'Fail to reject the null hypothesis.')

### 2-sided test hypothesis:

- State your hypothesis (H0 & H1)

Null Hypothesis (H0): There is no difference between the average cost per ride for a 2-minute versus a 5-minute wait during commute hour

Alternative Hypothesis (H1): There is a significant difference between the average cost per ride for a 2-minute versus a 5-minute wait during commute hour


- Set the level of significant (α)

assume the level of significant is 5% (at 95% confidence level)

- Drive test value by using proper statistical test

t-test

- Draw conclusion

The p-value is 0.2838, so we fail to reject the null hypothesis. This indicate that during commute hour, there's no difference for the average cost per ride for a 2-minute versus a 5-minute wait time

### non-commute

In [18]:
non_commute_2 = group_2_min_non_commute['cost_per_ride']
non_commute_5 = group_5_min_non_commute['cost_per_ride']

In [19]:
test_avg_cost_difference(non_commute_2, non_commute_5)

T-Statistic: 3.5404, P-Value: 0.0006


(3.540390260189291, 0.000599478655763849, 'Reject the null hypothesis')

### 2-sided test hypothesis:

- State your hypothesis (H0 & H1)

Null Hypothesis (H0): There is no difference between the average cost per ride for a 2-minute versus a 5-minute wait during non-commute hour

Alternative Hypothesis (H1): There is a significant difference between the average cost per ride for a 2-minute versus a 5-minute wait during non-commute hour


- Set the level of significant (α)

assume the level of significant is 5% (at 95% confidence level)

- Drive test value by using proper statistical test

t-test

- Draw conclusion

The p-value is 0.0006, so we reject the null hypothesis. This indicate that during non-commute hour, there is a significant difference between the average cost per ride for a 2-minute wait vs. a 5-minute wait

---------------------------------.

In [20]:
test_avg_cost_difference_1_side(non_commute_2, non_commute_5)

T-Statistic: 3.5404, P-Value: 0.0003


(3.540390260189291, 0.0002997393278819245, 'Reject the null hypothesis')

### 1-sided test hypothesis:

- State your hypothesis (H0 & H1)

Null Hypothesis (H0): The average cost per ride for a 2-minute waitis lower or equal to a 5-minute wait during non-commute hour

Alternative Hypothesis (H1): The average cost per ride for a 2-minute waitis significantly higher than a 5-minute wait during non-commute hour

- Set the level of significant (α)

assume the level of significant is 5% (at 95% confidence level)

- Drive test value by using proper statistical test

t-test

- Draw conclusion

As p-value is 0.0003, we Reject the null hypothesis, indicating that the average cost per ride is significantly higher for 2-minute wait times compared to 5-minute wait during non-commute hour

**Question 3(2 Points).** Write a function to test the significance of the difference between the ride cancellation rate for a 2-minute versus a 5-minute wait time. Write hypotheses and draw conclusions based on results	at 95% confidence interval.

In [21]:
from statsmodels.stats.proportion import proportions_ztest

In [22]:
group_2_min = df_raw[df_raw['wait_time'] == '2 mins']
group_5_min = df_raw[df_raw['wait_time'] == '5 mins']

In [23]:
df_raw.groupby('wait_time')['rider_cancellations_rate'].agg(['mean', 'std', 'count']).reset_index()

,wait_time,mean,std,count
0,2 mins,0.041216,0.006393,63
1,5 mins,0.048504,0.008354,63


In [24]:
## two_sided 
def test_cancel_rate_difference(df, group_2_min, group_5_min):


    # total cancellations & total rides 
    cancellations_2min = group_2_min['rider_cancellations'].sum()
    total_rides_2min = group_2_min['total_ride'].sum()

    cancellations_5min = group_5_min['rider_cancellations'].sum()
    total_rides_5min = group_5_min['total_ride'].sum()

    # Z-test
    stat, p_value = proportions_ztest([cancellations_2min, cancellations_5min], [total_rides_2min, total_rides_5min])


    if p_value < 0.05:
        conclusion = "Reject the null hypothesis: There is a significant difference in cancellation rates."
    else:
        conclusion = "Fail to reject the null hypothesis: No significant difference in cancellation rates."

    return stat, p_value, conclusion

In [25]:
test_cancel_rate_difference(df_raw, group_2_min, group_5_min)

(-12.372607519828444,
 3.676879568995279e-35,
 'Reject the null hypothesis: There is a significant difference in cancellation rates.')

### 2-sided test hypothesis:

- State your hypothesis (H0 & H1)

Null Hypothesis (H0): There is no difference between the cancellation rate for a 2-minute versus a 5-minute wait 

Alternative Hypothesis (H1): There is a significant difference between the cancellation rate for a 2-minute versus a 5-minute wait

- Set the level of significant (α)

assume the level of significant is 5% (at 95% confidence level)

- Drive test value by using proper statistical test

z-test

- draw conclusion

The p-value is close to 0 (3.676879568995279e-35), so we reject the null hypothesis. There is a significant difference in cancellation rates for a 2-minute versus a 5-minute wait

----------------------------------.

In [26]:
## one_sided 
def one_side_cancel_rate_difference(df, group_2_min, group_5_min):


    # total cancellations & total rides
    cancellations_2min = group_2_min['rider_cancellations'].sum()
    total_rides_2min = group_2_min['total_ride'].sum()

    cancellations_5min = group_5_min['rider_cancellations'].sum()
    total_rides_5min = group_5_min['total_ride'].sum()

    # Z-test
    stat, p_value = proportions_ztest([cancellations_2min, cancellations_5min], 
                                      [total_rides_2min, total_rides_5min],
                                     alternative = 'smaller')

    if p_value < 0.05:
        conclusion = "Reject the null hypothesis: The 2-min wait cancellation rate is significantly lower."
    else:
        conclusion = "Fail to reject the null hypothesis: The 2-min wait cancellation rate is greater or equal to the 5-min wait."

    return stat, p_value, conclusion

In [27]:
one_side_cancel_rate_difference(df_raw, group_2_min, group_5_min)

(-12.372607519828444,
 1.8384397844976394e-35,
 'Reject the null hypothesis: The 2-min wait cancellation rate is significantly lower.')

### 1-sided test hypothesis:

- State your hypothesis (H0 & H1)

Null Hypothesis (H0): The average cost per ride for a 5-minute wait time is higher than the one for a 2-minute wait time

Alternative Hypothesis (H1): The average cost per ride for a 5-minute wait time is less than or equal to the one for a 2-minute wait time

- Set the level of significant (α)

assume the level of significant is 5% (at 95% confidence level)

- Drive test value by using proper statistical test

Z-test

- draw conclusion

The p-value is close to 0 (1.8384397844976394e-35), so we reject the null hypothesis. This indicate that the cancellation rates for a 2-minute is significantly lower than the one for 5-minute wait

**Question 4(2 Points)**. Use the same  function to test the significance of the difference between the cancellation rate for a 2-minute versus a 5-minute wait time during commute hours. Write hypotheses and draw conclusions based on results at 95% confidence interval

In [28]:
group_2_min = df_raw[df_raw['wait_time'] == '2 mins']
group_5_min = df_raw[df_raw['wait_time'] == '5 mins']
group_2_min_commute = group_2_min[group_2_min['commute'] == True]
group_2_min_non_commute = group_2_min[group_2_min['commute'] == False]
group_5_min_commute = group_5_min[group_5_min['commute'] == True]
group_5_min_non_commute = group_5_min[group_5_min['commute'] == False]

In [29]:
group_2_min.groupby('commute')['rider_cancellations_rate'].agg(['mean', 'std', 'count']).reset_index()

,commute,mean,std,count
0,False,0.039820,0.005504,53
1,True,0.048614,0.005848,10


In [30]:
group_5_min.groupby('commute')['rider_cancellations_rate'].agg(['mean', 'std', 'count']).reset_index()

,commute,mean,std,count
0,False,0.045492,0.004647,53
1,True,0.064472,0.004379,10


In [31]:
## two-sided
def test_cancel_rate_difference(df, group_2_min, group_5_min):

    # total cancellations & total rides
    cancellations_2min = group_2_min['rider_cancellations'].sum()
    total_rides_2min = group_2_min['total_ride'].sum()

    cancellations_5min = group_5_min['rider_cancellations'].sum()
    total_rides_5min = group_5_min['total_ride'].sum()
    
    # Z-test
    stat, p_value = proportions_ztest([cancellations_2min, cancellations_5min], [total_rides_2min, total_rides_5min])

    if p_value < 0.05:
        conclusion = "Reject the null hypothesis: There is a significant difference in cancellation rates."
    else:
        conclusion = "Fail to reject the null hypothesis: No significant difference in cancellation rates."

    return stat, p_value, conclusion

In [32]:
test_cancel_rate_difference(df_raw, group_2_min_commute, group_5_min_commute)

(-10.335093413241797,
 4.889354936805803e-25,
 'Reject the null hypothesis: There is a significant difference in cancellation rates.')

### 2-sided test hypothesis:

- State your hypothesis (H0 & H1)

Null Hypothesis (H0): There is no difference between the cancellation rate for a 2-minute versus a 5-minute wait during commute hour

Alternative Hypothesis (H1): There is a significant difference between the cancellation rate for a 2-minute versus a 5-minute wait during commute hour

- Set the level of significant (α)

assume the level of significant is 5% (at 95% confidence level)

- Drive test value by using proper statistical test

z-test

- draw conclusion

The p-value is close to 0 (4.889354936805803e-25), so we reject the null hypothesis. There is a significant difference in cancellation rates for a 2-minute versus a 5-minute wait during commute hour

------------------------.

In [33]:
## one-sided
def one_side_cancel_rate_difference(df, group_2_min, group_5_min):


    # total cancellations & total rides
    cancellations_2min = group_2_min['rider_cancellations'].sum()
    total_rides_2min = group_2_min['total_ride'].sum()

    cancellations_5min = group_5_min['rider_cancellations'].sum()
    total_rides_5min = group_5_min['total_ride'].sum()

    # Z-test
    stat, p_value = proportions_ztest([cancellations_2min, cancellations_5min], 
                                      [total_rides_2min, total_rides_5min],
                                     alternative = 'smaller')

    if p_value < 0.05:
        conclusion = "Reject the null hypothesis: The 2-min wait cancellation rate is significantly lower during commute hour."
    else:
        conclusion = "Fail to reject the null hypothesis: The 2-min wait cancellation rate is greater or equal to the 5-min wait during commute hour."

    return stat, p_value, conclusion

In [34]:
one_side_cancel_rate_difference(df_raw, group_2_min, group_5_min)

(-12.372607519828444,
 1.8384397844976394e-35,
 'Reject the null hypothesis: The 2-min wait cancellation rate is significantly lower during commute hour.')

### 1-sided test hypothesis:

- State your hypothesis (H0 & H1)

Null Hypothesis (H0): The 2-min wait cancellation rate is greater or equal to 5-min wait cancellation rate during commute hour

Alternative Hypothesis (H1): The 2-min wait cancellation rate is lower than the 5-min wait during commute hour

- Set the level of significant (α)

assume the level of significant is 5% (at 95% confidence level)

- Drive test value by using proper statistical test

z-test

- draw conclusion

The p-value is close to 0 (1.8384397844976394e-35), so we reject the null hypothesis. This indicates that the 2-min wait cancellation rate is significantly lower than the 5-min wait cancellation rate during commute hour

**Question 5(2 Points)**.	Based on the data available to you and test analysis results, what would you recommend doing? Would you increase the wait time from 2 to 5 minutes in the six treatment cities of the launch experiment? (explain in words)

**From the result we have:**

In terms of average cost, there's a significant difference of average cost between 5-min wait and 2-min wait. But when we look into the commute hour and non-commute hour, we find out that during the commute hour, the average cost between 5-min wait and 2-min wait is actually no difference. In comparison, during the non-commute hour, there is a significant differnce of average cost between 2-min wait and 5-min wait. By running 1-sided hypothesis test, we find out that the average cost for 2-min wait is higher than the one for 5-min wait. 

In terms of cancellation rate, the 2-min wait cancellation rate is significantly lower than 5-min wait, for both all hours (commute + non-commute) and commute hour. 

Thus, I recommend to add *cummute hour* as our consideration. In other words, during the commute hour, as there is no difference of the cost between 2 waiting groups, but 2-min wait has lower cancellation rate. In this case, for higher efficiency and profit, we will assign more 2-min wait (decrease the wait time from 5 to 2 min). During the non-commute hour, we will assign more 2-min wait (increase the wait time from 2 to 5 min).

**In summary**, whether increase or decrease the wait time depends on whether the hour is commute hour or not. Thus, I will increase the wait time from 2 to 5 minutes during non-commute hour (in the six treatment cities of the launch experiment). However, I will decrease the wait time from 5 to 2 minutes during commute hour (in the six treatment cities of the launch experiment).